In [1]:
# example of loading the keras facenet model
from keras.models import load_model

In [2]:
# load the model
model = load_model('facenet_keras.h5')

In [3]:
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

0.1.0


In [4]:
from PIL import Image
import numpy as np

## DATA SET

### FACE DETECTION

In [5]:
# demonstrate face detection on 5 Celebrity Faces Dataset
from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN

In [6]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
	# load image from file
	image = Image.open(filename)
	# convert to RGB, if needed
	image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	# bug fix
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

In [7]:
# load images and extract faces for all images in a directory
def load_faces(directory):
	faces = list()
	# enumerate files
	for filename in listdir(directory):
		# path
		path = directory + filename
		# get face
		face = extract_face(path)
		# store
		faces.append(face)
	return faces

In [8]:
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
	X, y = list(), list()
	# enumerate folders, on per class
	for subdir in listdir(directory):
		# path
		path = directory + subdir + '/'
		# skip any files that might be in the dir
		#if not isdir(path):
			#continue
		# load all faces in the subdirectory
		faces = load_faces(path)
		# create labels
		labels = [subdir for _ in range(len(faces))]
		# summarize progress
		print('>loaded %d examples for class: %s' % (len(faces), subdir))
		# store
		X.extend(faces)
		y.extend(labels)
	return asarray(X), asarray(y)

In [15]:
# load train dataset
trainX, trainy = load_dataset('ttrain/')
print(trainX.shape, trainy.shape)

IndexError: list index out of range

In [ ]:
# load test dataset
testX, testy = load_dataset('val/')
print(testX.shape, testy.shape)

In [ ]:
import numpy as np
# save arrays to one file in compressed format
np.savez_compressed('masked-faces-dataset.npz', trainX, trainy, testX, testy)

In [ ]:
# load the face dataset
data = np.load('masked-faces-dataset.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

In [ ]:
# get the face embedding for one face
def get_embedding(model, face_pixels):
	# scale pixel values
	face_pixels = face_pixels.astype('float32')
	# standardize pixel values across channels (global)
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	# transform face into one sample
	samples = expand_dims(face_pixels, axis=0)
	# make prediction to get embedding
	yhat = model.predict(samples)
	return yhat[0]

In [ ]:
from numpy import expand_dims
# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
	embedding = get_embedding(model, face_pixels)
	newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)

In [ ]:
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
	embedding = get_embedding(model, face_pixels)
	newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)

In [ ]:
# save arrays to one file in compressed format
np.savez_compressed('masked-faces-embeddings.npz', newTrainX, trainy, newTestX, testy)

In [ ]:
# load dataset
data = np.load('masked-faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))

In [ ]:
from sklearn.preprocessing import Normalizer
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)

In [ ]:
from sklearn.preprocessing import LabelEncoder
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)

In [ ]:
from sklearn import svm

In [ ]:
# fit model
model = svm.SVC(kernel='linear')
model.fit(trainX, trainy)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
# predict
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)

In [ ]:
print ('Accuracy_train:', accuracy_score(trainy, yhat_train))
print ('F1 score_train:', f1_score(trainy, yhat_train,average='weighted'))
print ('Recall_train:', recall_score(trainy, yhat_train,
                              average='weighted'))
print ('Precision_train:', precision_score(trainy, yhat_train,
                                    average='weighted'))
print ('\n clasification report_train:\n', classification_report(trainy, yhat_train))
print ('\n confussion matrix_train:\n',confusion_matrix(trainy, yhat_train))

In [ ]:
print ('Accuracy_test:', accuracy_score(testy, yhat_test))
print ('F1 score_test:', f1_score(testy, yhat_test,average='weighted'))
print ('Recall_test:', recall_score(testy, yhat_test,
                              average='weighted'))
print ('Precision_test:', precision_score(testy, yhat_test,
                                    average='weighted'))
print ('\n clasification report_test:\n', classification_report(testy, yhat_test))
print ('\n confussion matrix_test:\n',confusion_matrix(testy, yhat_test))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(trainX, trainy)

In [ ]:
from sklearn.metrics import accuracy_score
# predict
yhat_train1 = knn.predict(trainX)
yhat_test1 = knn.predict(testX)

In [ ]:
print ('Accuracy_train:', accuracy_score(trainy, yhat_train1))
print ('F1 score_train:', f1_score(trainy, yhat_train,average='weighted'))
print ('Recall_train:', recall_score(trainy, yhat_train1,
                              average='weighted'))
print ('Precision_train:', precision_score(trainy, yhat_train1,
                                    average='weighted'))
print ('\n clasification report_train:\n', classification_report(trainy, yhat_train1))
print ('\n confussion matrix_train:\n',confusion_matrix(trainy, yhat_train1))

In [ ]:
print ('Accuracy_test:', accuracy_score(testy, yhat_test1))
print ('F1 score_test:', f1_score(testy, yhat_test1,average='weighted'))
print ('Recall_test:', recall_score(testy, yhat_test1,
                              average='weighted'))
print ('Precision_test:', precision_score(testy, yhat_test1,
                                    average='weighted'))
print ('\n clasification report_test:\n', classification_report(testy, yhat_test1))
print ('\n confussion matrix_test:\n',confusion_matrix(testy, yhat_test1))

In [ ]:
# develop a classifier for the 5 Celebrity Faces Dataset
from random import choice
from numpy import load
from numpy import expand_dims
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot
# load faces
data = load('masked-faces-dataset.npz')
testX_faces = data['arr_2']
# load face embeddings
data = load('masked-faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)
# test model on a random example from the test dataset
selection = choice([i for i in range(testX.shape[0])])
random_face_pixels = testX_faces[selection]
random_face_emb = testX[selection]
random_face_class = testy[selection]
random_face_name = out_encoder.inverse_transform([random_face_class])
# prediction for the face
samples = expand_dims(random_face_emb, axis=0)
yhat_class = model.predict(samples)
yhat_prob = model.predict_proba(samples)
# get name
class_index = yhat_class[0]
class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_class)
print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
print('Expected: %s' % random_face_name[0])
# plot for fun
pyplot.imshow(random_face_pixels)
title = '%s (%.3f)' % (predict_names[0], class_probability)
pyplot.title(title)
pyplot.show()